In [1]:
import os
import pandas as pd
import geopandas as gpd

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

## Downloading data through California Forest Observatory api

In [ ]:
#pip install cfo

In [ ]:
# import cfo
# #import gdal

In [ ]:
# forest_api = cfo.api()
# forest_api.authenticate()

In [ ]:
# forest_api.search(geography='California', metric='CanopyHeight')
# # msai4earth-esa/shapefiles_n_rasters/SB_geometry/SB_only.shp

In [ ]:
# itemid = forest_api.search(geography='SantaBarbaraCounty', metric='CanopyHeight')[0]
# itemid

In [ ]:
# lidar_file = os.path.join(os.getcwd(), 'santabarbara_lidar_2020.tif')

In [ ]:
# forest_api.download(itemid, lidar_file)

## Open lidar and create min,max,avg,diff lidar rasters

In [2]:
lidar_year = 2020

In [3]:
fp = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)+'.tif')
lidar_rast_r = rasterio.open(fp)

In [4]:
lsf.save_min_max_diff_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

lsf.save_avg_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

## Open points from given year and aoi

In [5]:
aoi = 'campus_lagoon'
aoi_year = 2020

pts_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                     aoi+'_points',
                     aoi+'_points_'+str(aoi_year)+'.csv')

## Obtain CRS from itemid and create geodataframe from points

In [6]:
from rasterio.crs import CRS

In [7]:
itemid = pd.read_csv(pts_fp).naip_id[0]
itemid

'ca_m_3411934_sw_11_060_20200521'

In [8]:
# accesing Azure storage using pystac client
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# search for naip scene where the pts were sampled from
search = catalog.search(
    collections=["naip"],
    ids = itemid
)
item = list(search.get_items())[0]
epsg_code = item.properties['proj:epsg']

In [9]:
pts = lsf.geodataframe_from_csv(pts_fp, CRS.from_epsg(epsg_code))
pts.head(3)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry
0,1,78,88,69,161,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238491.487 3810779.673)
1,1,85,89,70,162,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238490.963 3810789.330)
2,1,74,81,70,144,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238484.510 3810781.595)


## Sample LIDAR

In [16]:
pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)

In [17]:
lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)

In [18]:
lidar_fps = []
for tag in ['maxs_', 'mins_', 'avgs_']:
    lidar_fps.append(os.path.join(os.getcwd(),
                                 'SantaBarbaraCounty_lidar_'+str(lidar_year),
                                 'SantaBarbaraCounty_lidar_'+tag+ str(lidar_year)+'.tif'))
#lidar_fps

In [19]:
#fp = os.path.join(os.getcwd(),'lidar_maxs.tif')
maxs_rast_r = rasterio.open(lidar_fps[0])
max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

In [22]:
mins_rast_r = rasterio.open(lidar_fps[1])
min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

In [23]:
#fp = os.path.join(os.getcwd(),'lidar_avgs.tif')
avg_rast_r = rasterio.open(lidar_fps[2])
avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)

In [24]:
pts['lidar'] = lidar_samples
pts['max_lidar']= max_samples
pts['min_lidar'] = min_samples
pts['min_max_diff'] = pts.max_lidar - pts.min_lidar
pts['avg_lidar'] = avg_samples


In [25]:
pts.head(3)

,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar
0,1,78,88,69,161,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238491.487 3810779.673),2,6,0,6,2.555556
1,1,85,89,70,162,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238490.963 3810789.330),0,4,0,4,1.111111
2,1,74,81,70,144,2020,5,21,ca_m_3411934_sw_11_060_20200521,0,POINT (238484.510 3810781.595),4,6,0,6,3.111111


In [ ]:
ptslidar_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                     aoi+'_points',
                     aoi+'_points_lidar'+str(aoi_year)+'.csv')
pts.to_csv(ptslidar_fp)